In [1]:
import locale
locale.getdefaultlocale()

('en_US', 'UTF-8')

In [2]:
!pip install transformers # need to install from github
!pip install datasets loralib sentencepiece
!pip install bitsandbytes accelerate xformers einops
!pip install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/

In [3]:

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "bn22/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = transformers.BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
model_id,
trust_remote_code=True,
quantization_config=bnb_config,
device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
model_id,
)


Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install flask
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!pip install -U flask-cors

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=ad46b1b47ff2f801fe1768f198d808f9dbf2f0ded289a47ee35a5b65d85d5122
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [5]:
!ngrok authtoken 2Wk8jK9HjkDjslpYv00lNy00WmN_7p5sQA8Fwb7yzz8D25Py7

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [34]:
def process_prompt(image_desc):

  prompt = f"""<s>[INST]
You are a social media content creator.
You have a deep understanding on how to produce content given a trend.
You will be provided with a image description and list of trends that are trending right now.
Use the image description and the list of trends to assist user on how they can use that idea and make a video.
Properly format your response with each trend. Use styling, new lines and emjois where necessary.
Strictyl use the response template to generate ideas for all the trends in the list.

User Input Template:
Image Description: {image_desc}
Trend List:
1: Pair an audio with a caption that highlights a risk you’ve recently taken and let the delusion soar.
2: The perfect opportunity to showcase a killer fit.
3: Highlight an unexpected moment or a relatable struggle.

Response Template Example in markdown for each item in the list of trends:
```
## Trend:
trend from the list of trends

## Content Idea:\n
detailed idea to shoot short video from the image description relating to the trend. Derive the idea ONLY from the image description

## Recording Ideas:\n
help the creator to tell an appealing story step by step with your creativity while being safe.
- recording idea 1
- recording idea 2
```
[/INST]</s>
Here are some video ideas for the given image description and trend list:
"""

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
  model_inputs = encodeds.to('cuda')
  generated_ids = model.generate(**encodeds,max_new_tokens=3000,do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0]

In [32]:
print(process_prompt("there is a man standing in a narrow walkway between two trees"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]
You are a social media content creator.
You have a deep understanding on how to produce content given a trend.
You will be provided with a image description and list of trends that are trending right now.
Use the image description and the list of trends to assist user on how they can use that idea and make a video.
Properly format your response with each trend. Use styling, new lines and emjois where necessary.
Strictyl use the response template to generate ideas for all the trends in the list.

User Input Template:
Image Description: there is a man standing in a narrow walkway between two trees
Trend List:
1: Pair an audio with a caption that highlights a risk you’ve recently taken and let the delusion soar.
2: The perfect opportunity to showcase a killer fit.
3: Highlight an unexpected moment or a relatable struggle.

Response Template Example in markdown for each item in the list of trends:
```
# [format this in markdown heading 2] Trend:
trend from the list of trends

# [

In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin
from json import loads

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

run_with_ngrok(app)

@app.route("/generate/",methods=['POST'])
@cross_origin()
def home():
  data = request.json;
  prompt = data['prompt']

  print(prompt)

  if len(prompt) == 0:
    return {
        "message": "Text is empty",
        "code": 400
    }, 400

  generated_text = process_prompt(prompt)
  return {
      "generated_text": generated_text,
      "code": 200
  },200
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://854a-34-126-125-2.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


araffes a man and a woman are kissing in the dark


...

INFO:werkzeug:127.0.0.1 - - [15/Oct/2023 01:57:53] "POST /generate/ HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


araffe man with sunglasses and beard standing in front of a bush of flowers


INFO:werkzeug:127.0.0.1 - - [15/Oct/2023 02:00:14] "POST /generate/ HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


araffes a man and a woman are kissing in the dark


INFO:werkzeug:127.0.0.1 - - [15/Oct/2023 02:02:36] "POST /generate/ HTTP/1.1" 200 -


In [9]:
a = {"a": 1}
a.get('a')

1